In [1]:
#Importazioni Librerie
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
#Importazione Dataset
file_path = r"C:\Users\JoaquimFrancalanci\OneDrive - ITS Angelo Rizzoli\Desktop\MachineLearning\shopping_behavior_updated.csv"
df = pd.read_csv(file_path)

In [3]:
# Visualizzazione delle prime righe del dataset
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [4]:
#Prima Visione Dataset
print(df.describe())

       Customer ID          Age  Purchase Amount (USD)  Review Rating  \
count  3900.000000  3900.000000            3900.000000    3900.000000   
mean   1950.500000    44.068462              59.764359       3.749949   
std    1125.977353    15.207589              23.685392       0.716223   
min       1.000000    18.000000              20.000000       2.500000   
25%     975.750000    31.000000              39.000000       3.100000   
50%    1950.500000    44.000000              60.000000       3.700000   
75%    2925.250000    57.000000              81.000000       4.400000   
max    3900.000000    70.000000             100.000000       5.000000   

       Previous Purchases  
count         3900.000000  
mean            25.351538  
std             14.447125  
min              1.000000  
25%             13.000000  
50%             25.000000  
75%             38.000000  
max             50.000000  


In [5]:
#Shape del dataset
print(f"Il dataset ha una dimensione {df.shape}\n")

Il dataset ha una dimensione (3900, 18)



In [6]:
#Column  Non-Null Count  Dtype 
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3900 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [7]:
# Identificazione delle righe duplicate
duplicated_rows = df[df.duplicated(keep=False)]

# Numero di righe duplicate
print(f"Le righe duplicate sono {duplicated_rows.shape[0]}")

Le righe duplicate sono 0


In [8]:
df = df.drop(columns=['Customer ID']) # Rimuovere il campo 'Customer ID'
X = df.drop(columns=['Purchase Amount (USD)'])
y = df['Purchase Amount (USD)']


In [9]:
#Variabili Categoriche
categorical_cols = X.select_dtypes(include=['object']).columns
print("Categorical columns:", categorical_cols)

for col in categorical_cols:
    X[col] = X[col].astype('category').cat.codes

Categorical columns: Index(['Gender', 'Item Purchased', 'Category', 'Location', 'Size', 'Color',
       'Season', 'Subscription Status', 'Shipping Type', 'Discount Applied',
       'Promo Code Used', 'Payment Method', 'Frequency of Purchases'],
      dtype='object')


In [10]:
# Verifica Conversione
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     3900 non-null   int64  
 1   Gender                  3900 non-null   int8   
 2   Item Purchased          3900 non-null   int8   
 3   Category                3900 non-null   int8   
 4   Location                3900 non-null   int8   
 5   Size                    3900 non-null   int8   
 6   Color                   3900 non-null   int8   
 7   Season                  3900 non-null   int8   
 8   Review Rating           3900 non-null   float64
 9   Subscription Status     3900 non-null   int8   
 10  Shipping Type           3900 non-null   int8   
 11  Discount Applied        3900 non-null   int8   
 12  Promo Code Used         3900 non-null   int8   
 13  Previous Purchases      3900 non-null   int64  
 14  Payment Method          3900 non-null   

In [11]:
X.head()

,Age,Gender,Item Purchased,Category,Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,55,1,2,1,16,0,7,3,3.1,1,1,1,1,14,5,3
1,19,1,23,1,18,0,12,3,3.1,1,1,1,1,2,1,3
2,50,1,11,1,20,2,12,1,3.1,1,2,1,1,23,2,6
3,21,1,14,2,38,1,12,1,3.5,1,3,1,1,49,4,6
4,45,1,2,1,36,1,21,1,2.7,1,2,1,1,31,4,0


In [12]:
# Divisione dei dati in training e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(f"Shape of training data: {X_train.shape}")

# Inizializziamo il modello
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)
rf_model.fit(X_train, y_train) # Addestriamo il modello
y_pred = rf_model.predict(X_test)



Shape of training data: (3120, 16)


In [13]:
#media Y test & Y pred
media_yT = y_test.mean()
print(f"La media dell' y test è: {round(media_yT,3)}")

media_yP = y_pred.mean()
print(f"La media dell' y predict è: {round(media_yP,3)}")

La media dell' y test è: 59.899
La media dell' y predict è: 59.762


In [14]:
#Deviazione standard y_test & y_pred
std_yT = y_test.std()
print(f"La deviazione standard dell' y_test è: {round(std_yT,3)}")

std_yP = y_pred.std()
print(f"La deviazione standard dell' y_pred è: {round(std_yP,3)}")

La deviazione standard dell' y_test è: 23.358
La deviazione standard dell' y_pred è: 4.593


In [15]:
#Deviazione standard Y Test & Y Pred
std_y = y.std()
print(f"La deviazione standard dell' y è: {round(std_y,3)}")

std_yP = y_pred.std()
print(f"La deviazione standard dell' ypred è: {round(std_yP,3)}")

La deviazione standard dell' y è: 23.685
La deviazione standard dell' ypred è: 4.593
